In [1]:
from get_images_and_labels import *
import numpy as np
from utils import denoise_image, lowe_filter
from evaluation import remove_background_and_eval, mapk
import cv2

## Get the Data

In [2]:
labels = get_query_set_labels(os.getcwd(), "qsd1_w4")

qs = get_query_set_images(os.getcwd(), "qsd1_w4")
bbdd = get_museum_dataset(os.getcwd())

qs = [denoise_image(img) for img in qs]
qs = remove_background_and_eval(qs, os.getcwd(), "qsd1_w4", True)

Removing backgrounds!


C:\Users\myaza\Week4\evaluation.py:418: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = 2*precision*recall/(precision+recall)


Precision: 0.8979281985506441 Recall: 0.9374705793047161 F1: nan


In [59]:
def match_algorithm(desc1, desc2, method="BF", k=2):
    
    if method == "flann":
    
        index = dict(algorithm=0, trees=5)
        search = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index, search)
        matches = flann.knnMatch(desc1, desc2, k)
        
    elif method=="BF":
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)
        matches = bf.knnMatch(desc1, desc2, k=k)      
    
    return lowe_filter(matches)

In [69]:
def ORB(img, keypoint_size):

    orb = cv2.ORB_create(keypoint_size)
    return orb.detectAndCompute(img, None)

def DAISY(img, keypoint_size):


    daisy = cv2.xfeatures2d.DAISY_create(keypoint_size)
    return daisy.detectAndCompute(img, None)

def SIFT(img, keypoint_size):
    
    sift = cv2.SIFT_create(keypoint_size)
    return sift.detectAndCompute(img, None)

local_descs = {
    "SIFT": SIFT,
    "ORB": ORB,
    "DAISY": DAISY
}

def get_local_descriptors(method, img, keypoint_size):
    return local_descs[method](img, keypoint_size)

# Create SIFT desciptors for museum and query set

In [70]:
def calc_local_descs_matches(desc_method, match_type, keypoint_size):
    bbdd_descs = []

    for img in bbdd:
        _, descs = get_local_descriptors(desc_method, img, keypoint_size)
        bbdd_descs.append(descs)
        
    img_matches = []

    for i, img in enumerate(qs):

        painting_list = []
        if not isinstance(img, list):
            img = [img]

        for painting in img:

            _, img_desc = get_local_descriptors(desc_method, painting, keypoint_size)

            if img_desc is None:
                img_desc = []

            else:
                bbdd_match = []
                for bbdd_desc in bbdd_descs:
                    bbdd_match.append(match_algorithm(bbdd_desc, img_desc, method=match_type))

                painting_list.append(bbdd_match)

        img_matches.append(painting_list)
        
    return img_matches

In [71]:
def get_map_for_local_descs(img_matches, match_threshold):

    final_match = []

    for img in img_matches:

        img_match = []

        for painting in img:
            painting_match = []

            for matches in painting:
                painting_match.append(len(matches))

            if np.max(np.array(painting_match)) < match_threshold:
                img_match.append([-1])
            else:
                img_match.append(np.argsort(np.array(painting_match))[::-1][:10])

        final_match.append(img_match)

    return round(mapk(labels, final_match, 1), 3)

## SIFT 1000-150 0.48 mAP Flann

## ORB 1250-50 0.63 mAP BF

In [93]:
for ks in [500, 750, 1000, 1250, 1500]:
    img_matches = calc_local_descs_matches("SIFT", "BF", ks)  
    
    for thres in [50, 75, 100, 125, 150, 175, 200]:
        map = get_map_for_local_descs(img_matches, thres)
        
        print("For Keypoint Size:", ks, "and threshold:", thres, "mAP is", map)

KeyboardInterrupt: 

In [94]:
for ks in [500, 750, 1000, 1250, 1500]:
    img_matches = calc_local_descs_matches("ORB", "BF", ks)  
    
    for thres in [50, 75, 100, 125, 150, 175, 200]:
        map = get_map_for_local_descs(img_matches, thres)
        
        print("For Keypoint Size:", ks, "and threshold:", thres, "mAP is", map)

For Keypoint Size: 500 and threshold: 50 mAP is 0.371
For Keypoint Size: 500 and threshold: 75 mAP is 0.343
For Keypoint Size: 500 and threshold: 100 mAP is 0.343
For Keypoint Size: 500 and threshold: 125 mAP is 0.343
For Keypoint Size: 500 and threshold: 150 mAP is 0.314
For Keypoint Size: 500 and threshold: 175 mAP is 0.314
For Keypoint Size: 500 and threshold: 200 mAP is 0.314
For Keypoint Size: 750 and threshold: 50 mAP is 0.429
For Keypoint Size: 750 and threshold: 75 mAP is 0.371
For Keypoint Size: 750 and threshold: 100 mAP is 0.343
For Keypoint Size: 750 and threshold: 125 mAP is 0.343
For Keypoint Size: 750 and threshold: 150 mAP is 0.343
For Keypoint Size: 750 and threshold: 175 mAP is 0.343
For Keypoint Size: 750 and threshold: 200 mAP is 0.314
For Keypoint Size: 1000 and threshold: 50 mAP is 0.486
For Keypoint Size: 1000 and threshold: 75 mAP is 0.4
For Keypoint Size: 1000 and threshold: 100 mAP is 0.4
For Keypoint Size: 1000 and threshold: 125 mAP is 0.4
For Keypoint Size: